# Анализ базы данных сервиса для чтения книг

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;

- `author_id` — идентификатор автора;

- `title` — название книги;

- `num_pages` — количество страниц;

- `publication_date` — дата публикации книги;

- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;

- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;

- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;

- `book_id` — идентификатор книги;

- `username` — имя пользователя, оставившего оценку;

- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;

- `book_id` — идентификатор книги;

- `username` — имя пользователя, написавшего обзор;

- `text` — текст обзора.

# Цели исследования:

Проанализировать информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. Сформулировать ценностное предложение для нового продукта

# Наша задача:

- Посчитать, сколько книг вышло после 1 января 2000 года;

- Для каждой книги посчитать количество обзоров и среднюю оценку;

- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;

- Определить автора с самой высокой средней оценкой книг — учтём только книги с 50 и более оценками;

- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
import pandas as pd
from sqlalchemy import create_engine 

***Установим параметры***

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

***Сохраняем коннектор***

In [3]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [4]:
books = ''' SELECT * FROM books; ''' 
books = pd.io.sql.read_sql(books, con = engine)

In [5]:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [7]:
books.duplicated().sum()

0

In [8]:
books.isna().sum()

book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64

In [9]:
reviews = ''' SELECT * FROM reviews; ''' 
reviews = pd.io.sql.read_sql(reviews, con = engine)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


In [10]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [11]:
reviews.duplicated().sum()

0

In [12]:
reviews.isna().sum()

review_id    0
book_id      0
username     0
text         0
dtype: int64

In [13]:
ratings = ''' SELECT * FROM ratings; ''' 
ratings = pd.io.sql.read_sql(ratings, con = engine)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [14]:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [15]:
ratings.duplicated().sum()

0

In [16]:
ratings.isna().sum()

rating_id    0
book_id      0
username     0
rating       0
dtype: int64

In [17]:
publishers = ''' SELECT * FROM publishers; ''' 
publishers = pd.io.sql.read_sql(publishers, con = engine)
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [18]:
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [19]:
publishers.duplicated().sum()

0

In [20]:
publishers.isna().sum()

publisher_id    0
publisher       0
dtype: int64

In [21]:
authors = ''' SELECT * FROM authors; ''' 
authors = pd.io.sql.read_sql(authors, con = engine)
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [22]:
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [23]:
authors.duplicated().sum()

0

In [24]:
authors.isna().sum()

author_id    0
author       0
dtype: int64

***Пропусков и дубликатов нигде нет***

# Задание №1:

# Посчитайте, сколько книг вышло после 1 января 2000 года

In [25]:
books_count = ''' 
SELECT COUNT(*) 
FROM books
WHERE publication_date > '2000-1-1'; 
''' 
question1 = pd.io.sql.read_sql(books_count, con = engine)
print(f'После 1 января 2000 года вышло {question1.iloc[0,0]} книг')

После 1 января 2000 года вышло 819 книг


***Вывод:***

- После 1го января 2000 года вышло 819 книг

# Задание №2:

# Для каждой книги посчитайте количество обзоров и среднюю оценку

In [26]:
book_rating = ''' 

SELECT 
    books.title AS name_book,
    books.book_id AS id,
    rev.n_rev AS reviews,
    rat.avg_rat AS avg_rating
FROM books LEFT JOIN 
                (SELECT
                    book_id,
                    AVG(rating) AS avg_rat
                FROM
                    ratings
                GROUP BY 
                    book_id
                ) AS rat ON rat.book_id = books.book_id
        LEFT JOIN
                (SELECT 
                    book_id,
                    COUNT(review_id) AS n_rev
                FROM
                    reviews
                GROUP BY 
                    book_id
                ) AS rev ON rev.book_id = books.book_id
                
; ''' 

question2 = pd.io.sql.read_sql(book_rating, con = engine)
question2

,name_book,id,reviews,avg_rating
0,The Body in the Library (Miss Marple #3),652,2.0,4.500000
1,Galápagos,273,2.0,4.500000
2,A Tree Grows in Brooklyn,51,5.0,4.250000
3,Undaunted Courage: The Pioneering First Missio...,951,2.0,4.000000
4,The Prophet,839,4.0,4.285714
...,...,...,...,...
995,Alice in Wonderland,64,4.0,4.230769
996,A Woman of Substance (Emma Harte Saga #1),55,2.0,5.000000
997,Christine,148,3.0,3.428571
998,The Magicians' Guild (Black Magician Trilogy #1),790,2.0,3.500000


***Обзоры в колонке reviews, средняя оценка в колонке avg_rating***

# Задание №3:

# Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [27]:
publishers = ''' 
SELECT
    Pub.publisher_id,
    Pub.publisher AS name_publisher,
    Pub.count_books
FROM
    (SELECT 
        books.publisher_id AS publisher_id,
        COUNT(books.book_id) AS count_books,
        publishers.publisher AS publisher 
    FROM 
        books 
        LEFT JOIN publishers ON books.publisher_id=publishers.publisher_id

    WHERE 
        num_pages>50
    GROUP BY
        books.publisher_id,
        publishers.publisher
    ORDER BY 
        count_books DESC) AS Pub
LIMIT 1
; ''' 

question3 = pd.io.sql.read_sql(publishers, con = engine)
question3

,publisher_id,name_publisher,count_books
0,212,Penguin Books,42


***Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books, издательство выпустило 42 книги***

# Задание №4:

# Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [28]:
author_rating = ''' 
SELECT 
    avg.author,
    AVG(avg.avg_rating) AS rating_aut
FROM
        (SELECT 
            authors.author AS author
           , AVG(ratings.rating) AS avg_rating
        FROM books
            LEFT JOIN authors ON books.author_id = authors.author_id
            LEFT JOIN ratings ON books.book_id = ratings.book_id
        GROUP BY 
             books.book_id
             , authors.author_id
        HAVING
            COUNT(ratings.rating_id) >=50 
        ) AS avg 
GROUP BY    
    avg.author
ORDER BY
    rating_aut DESC
LIMIT 5
    
; ''' 

question4 = pd.io.sql.read_sql(author_rating, con = engine)
question4

,author,rating_aut
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


***Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré***

# Задание №5:

# Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [29]:
count_reviews = ''' 

SELECT
    AVG(avg.rev)
FROM
    (SELECT
        username,
        COUNT(review_id) AS rev
    FROM
        reviews
    WHERE 
        username IN (
                SELECT 
                    username
                FROM 
                    ratings
                GROUP BY 
                    username
                HAVING 
                    COUNT(rating_id) > 50
                    )
    GROUP BY 
        username
    ) AS avg
; ''' 

questions5 = pd.io.sql.read_sql(count_reviews, con = engine)
questions5

,avg
0,24.333333


***Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок равно 24***